In [1]:
from dotenv import load_dotenv

load_dotenv()

from openai import OpenAI

openai = OpenAI()

response = openai.chat.completions.create(
    model='openai.gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': 'Hello?'},
    ],
).choices[0].message.content

print(f'GPT-40-mini:\n\n{response}\n\n')



GPT-40-mini:

Hello! How can I assist you today?


